##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
import sys
sys.path.append('../') # 因为func与ipynb位于同一个目录下，所以要往上一层路径索引
from func.env import setProxy
cb=setProxy()
proxy=cb['proxy']
region=cb['region']

## <i class="fa fa-arrow-down"></i> 下载并安装InvokeAI
手动安装方法出处：https://invoke-ai.github.io/InvokeAI/installation/020_INSTALL_MANUAL/#conda-method

In [ ]:
import os

invokeai_dir=os.path.join(root_dir,'InvokeAI')

if not os.path.exists(invokeai_dir):
    !$proxy &&\
    cd $root_dir &&\
    git clone https://github.com/invoke-ai/InvokeAI.git
else:
    print(f'{invokeai_dir}已经存在，无需克隆')

!cd $invokeai_dir &&\
ln -sf environments-and-requirements/environment-lin-cuda.yml environment.yml

env_yml_path=os.path.join(invokeai_dir,'environment.yml')
linux_env_yml_path=os.path.join(invokeai_dir,'environments-and-requirements/environment-lin-cuda.yml')

if os.path.realpath(env_yml_path)==linux_env_yml_path:
    print(f'environment.yml已链接到 InvokeAI 根目录中，并且它指向 {linux_env_yml_path}')

    import subprocess
    # Use the subprocess module to run the "conda env list" command
    result = subprocess.run(["conda", "env", "list"], stdout=subprocess.PIPE)
    # Get the output of the command as a string and split it into a list of lines
    output = result.stdout.decode("utf-8").split("\n")

    # Check if the "invokeai" environment is in the list
    if "invokeai" in output:
        # If it is, activate the environment
        # 通过conda构建适合的python环境
        !$proxy &&\
        cd $invokeai_dir &&\
        conda env update
    else:
        print('已经存在名为invokeai的虚拟环境')
        # conda激活invokeai环境，且运行配置.py文件
        !$proxy &&\
        source activate &&\
        conda activate invokeai &&\
        cd $invokeai_dir &&\
        python scripts/configure_invokeai.py -y

## <i class="fa fa-wrench"></i> 安装 PyPatchMatch
pypatchmatch 是一个用于修复图像的 Python 模块。不需要运行InvokeAI，但它大大提高了修复和修复的质量，推荐使用。
不幸的是，它是一个 C++ 优化模块，安装可能有些困难。本指南将引导您完成这些步骤。
原文地址:https://invoke-ai.github.io/InvokeAI/installation/060_INSTALL_PATCHMATCH/#macintosh

### 安装必要工具

In [ ]:
base_args = [
    proxy,'&&',
    'source activate','&&',
    'conda activate invokeai','&&',
    f'cd {invokeai_dir}','&&'
]

def check_patchmatch_installed():
    install_pypatch_args = base_args[:] + [
        'python -c "from patchmatch import patch_match as pm;print(pm.patchmatch_available)"'
    ]
    install_pypatch_cmd = (" ").join(install_pypatch_args)
    pypatch_cb = !$install_pypatch_cmd
    if pypatch_cb[0] != 'True':
        print('pypatchmatch还没安装，即将安装')
        return False
    else:
        print('pypatchmatch已经安装，跳过安装')
        return True

patchmatch_installed = False
patchmatch_installed = check_patchmatch_installed()
print(patchmatch_installed)

if not patchmatch_installed:
    import subprocess

    libraries_to_install = [
    ]

    tools_to_install = [
        'build-essential',
        'python3-opencv',
        'libopencv-dev'
    ]

    installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

    # 尝试安装所有未安装的库
    for library in libraries_to_install:
        if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
            print(f"【 {library} 】已经安装，跳过安装")
            continue
        elif not any(library in installed_library for installed_library in installed_libraries):
            !pip install $library
        else:
            print(f"【 {library} 】已经安装，跳过安装")

    # 尝试安装所有未安装的工具
    update_needed = False
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            update_needed = True
            break
        else:
            print(f"【 {tool} 】已经安装，跳过安装")
    if update_needed:
        !apt-get update
        for tool in tools_to_install:
            exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
            if exit_code != 0:
                !apt-get install -y $tool

### 实装pypatchmatch并导入该模块进行检验

In [ ]:
if not patchmatch_installed:
    
    import os

    pkgconfig_dir='/usr/lib/x86_64-linux-gnu/pkgconfig/'
    if os.path.exists(pkgconfig_dir):
        !cd $pkgconfig_dir &&\
        ln -sf opencv4.pc opencv.pc

    #检查 opencv.pc 是否指向 opencv4.pc
    opencv_pc_path = os.path.join(pkgconfig_dir,'opencv.pc')
    opencv_pc4_path = os.path.join(pkgconfig_dir,'opencv4.pc')

    if os.path.realpath(opencv_pc_path)==opencv_pc4_path:
        print('opencv包配置文件的命名已修复')

    # 激活 invokeai 的环境并安装 pypatch
    install_pypatch_args = base_args[:] + [
        'pip install "git+https://github.com/invoke-ai/PyPatchMatch@0.1.3#egg=pypatchmatch"'
    ]

    install_pypatch_cmd = (" ").join(install_pypatch_args)

    !$install_pypatch_cmd

    check_installed_args = base_args[:] + [
        'python -c "from patchmatch import patch_match"'
    ]

    check_installed_cmd = (" ").join(check_installed_args)

    !$check_installed_cmd

## <i class="fa fa-eye"></i> 以WebUI的形式去启动InvokeAI

In [ ]:
invoke_outputs_dir='/root/autodl-nas/invoke_outputs'

git_pull_args = base_args[:] + ['git pull']
git_pull_cmd = (" ").join(git_pull_args)

git_pull_info = !$git_pull_cmd

if 'Already up to date.' not in git_pull_info:
    print('准备更新...')
    final_args.append([
        'conda env update','&&',
        'python scripts/configure_invokeai.py -y --skip-sd-weights'
    ])
else:
    print('已经更新到最新的，直接启动')
    
'''
在 Python 中，可以使用以下几种方法来拼接两个列表：
使用 + 操作符：new_list = list1 + list2。这会创建一个新的列表，其中包含原始两个列表中的所有元素。
使用内置函数 extend()：list1.extend(list2)。这会将 list2 中的所有元素添加到 list1 的末尾。
使用内置函数 sum()：new_list = sum([list1, list2], [])。这会将两个列表拼接起来，并将结果保存到新的列表中。
'''

final_args = base_args[:]
final_args += [
    'python scripts/invoke.py',
    '--web',
    '--port 6006',
    f'--outdir {invoke_outputs_dir}'
    ]

final_cmd = (" ").join(final_args)
!$final_cmd

## <i class="fa fa-arrow-down"></i>【即将废弃|无法安装成功】下载InvokeAI安装器

In [ ]:
# import requests
# import os
# import subprocess

# # Replace {OWNER} and {REPO} with the owner and repository name respectively
# url = f"https://api.github.com/repos/invoke-ai/InvokeAI/releases/latest"

# # Make the GET request to the GitHub API
# response = requests.get(url)

# # The response is a JSON object, which we can convert to a Python dictionary
# release_info = response.json()

# # print(release_info)
# # Access the desired information from the dictionary
# release_name = release_info["name"]
# tag_name = release_info["tag_name"]

# release_assets = release_info["assets"]

# invokeai_linux_installer_url=None

# for i in release_assets:
#     if 'linux' in i['browser_download_url']:
#         invokeai_linux_installer_url=i['browser_download_url']
        
# if invokeai_linux_installer_url:
#     file_name = os.path.basename(invokeai_linux_installer_url)
#     local_dir=os.path.join(root_dir,file_name)
#     installer_dir=os.path.join(root_dir,'InvokeAI-Installer')
#     install_sh_path=os.path.join(installer_dir,'install.sh')
    
#     if not os.path.exists(local_dir):
#         #下载安装器压缩包
#         !$proxy && wget -O $local_dir $invokeai_linux_installer_url
#     else:
#         print(f'本地已经存在 {file_name}，不进行下载')
        
#     if not os.path.exists(installer_dir):
#         #解压压缩包
#         !unzip -o $local_dir -d $root_dir
#     else:
#         print(f'本地已经存在 {installer_dir}，不进行解压')
        
#     if os.path.exists(install_sh_path):
#         print('正准备安装')

